In [2]:
import pymongo
from arxiv_api import arxiv_api
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.arxiv_papers

papers = db.papers
authors = db.authors

In [3]:
import time

In [15]:
for p in incomplete_papers():
    print(p)
    break

{'_id': ObjectId('591d038babac9b5de987c6e0'), 'id': 'http://arxiv.org/abs/0705.0998', 'title': 'The alternating sign matrix polytope', 'summary': '  We define the alternating sign matrix polytope as the convex hull of nxn\nalternating sign matrices and prove its equivalent description in terms of\ninequalities. This is analogous to the well known result of Birkhoff and von\nNeumann that the convex hull of the permutation matrices equals the set of all\nnonnegative doubly stochastic matrices. We count the facets and vertices of the\nalternating sign matrix polytope and describe its projection to the\npermutohedron as well as give a complete characterization of its face lattice\nin terms of modified square ice configurations. Furthermore we prove that the\ndimension of any face can be easily determined from this characterization.\n', 'authors': ['Striker']}


In [4]:
def incomplete_papers():
    for p in papers.find({ "title_detail" : { "$exists" : False } }):
        if 'id' not in p:
            papers.delete_one({'_id': p['_id']})
        else:
            yield p

In [5]:
def get_batch(gen, n):
    while True:
        yield [gen.__next__() for i in range(n)]

In [29]:
import re
id_mask = re.compile("^(.*)/abs/(.*)v?([0-9]*)$")
version_mask = re.compile("^([a-z\-]+/)?[0-9\.]+v([0-9]+)$")


match = id_mask.match("http://arxiv.org/abs/nucl-th/9411004v1")
id_string = match.group(2)
print(id_string)

match = version_mask.match(id_string)
print(match.group(2))

nucl-th/9411004v1
1


In [48]:
def both_versions(batch):
    for p in batch:
        if 'id' in p:
            doc_id = id_mask.match(p['id']).group(2)
            try:
                version = int(version_mask.match(doc_id).group(2))
                yield from ['v'.join(doc_id.split('v')[:-1] + [str(v)]) for v in range(1, version + 1)]
            except Exception as e:
                yield doc_id

In [53]:
count = 0
for batch in get_batch(incomplete_papers(), 100):
    try: 
        id_list = [id_string for id_string in both_versions(batch)]
        results = arxiv_api.query(id_list=id_list, start=None, max_results=1000)
        for r in results:
            del r['title_detail']['base']
            del r['summary_detail']['base']
            papers.update_one({"id":r['id']}, {"$set": r}, upsert=True)
            count += 1
        if count % 1000 == 0:
            print(count, end=', ')
    except Exception as e:
        print(id_list, e)
    for p in batch:
        papers.delete_one({"_id": p['_id']})

/home/ge/anaconda3/envs/deep-learning/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: generator 'get_batch' raised StopIteration
  from ipykernel import kernelapp as app


In [22]:
%reload_ext arxiv_api

In [7]:
results = arxiv_api.query(id_list=['0705.0998', '0705.0999'])

{'affiliation': 'School of Mathematics and Statistics, University of St Andrews, UK',
 'arxiv_comment': '21 pages, 16 figures Accepted for ApJ',
 'arxiv_primary_category': {'scheme': 'http://arxiv.org/schemas/atom',
  'term': 'astro-ph'},
 'arxiv_url': 'http://arxiv.org/abs/0705.1097v1',
 'author': 'A. W. Hood',
 'author_detail': {'name': 'A. W. Hood'},
 'authors': ['K. Galsgaard',
  'V. Archontis',
  'F. Moreno-Insertis',
  'A. W. Hood'],
 'doi': '10.1086/519756',
 'guidislink': True,
 'id': 'http://arxiv.org/abs/0705.1097v1',
 'journal_reference': 'Astrophys.J.666:516-531,2007',
 'links': [{'href': 'http://dx.doi.org/10.1086/519756',
   'rel': 'related',
   'title': 'doi',
   'type': 'text/html'},
  {'href': 'http://arxiv.org/abs/0705.1097v1',
   'rel': 'alternate',
   'type': 'text/html'},
  {'href': 'http://arxiv.org/pdf/0705.1097v1',
   'rel': 'related',
   'title': 'pdf',
   'type': 'application/pdf'}],
 'pdf_url': 'http://arxiv.org/pdf/0705.1097v1',
 'published': '2007-05-08T13: